# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-12.15,85,20,9.26,CA,1712225329
1,1,haiku-pauwela,20.9219,-156.3051,22.25,84,40,8.23,US,1712225329
2,2,lazaro cardenas,17.9583,-102.2000,20.29,86,7,1.72,MX,1712225197
3,3,port mathurin,-19.6833,63.4167,27.09,80,96,8.64,MU,1712225329
4,4,pudozh,61.8099,36.5345,-6.70,89,100,5.81,RU,1712225329


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
map_display = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_display

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_condition_1 = city_data_df['Cloudiness'] <1
filtered_condition_2 = city_data_df['Humidity'] < 60
filtered_condition_3 = city_data_df['Humidity'] > 30
filtered_condition_4 = city_data_df['Wind Speed'] < 8

filtered_data_df = city_data_df[filtered_condition_1 & filtered_condition_2 & filtered_condition_3 & filtered_condition_4]
# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
vacation_data_df = pd.DataFrame(filtered_data_df)
vacation_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,port elizabeth,-33.9180,25.5701,31.16,40,0,2.06,ZA,1712225217
50,50,alice springs,-23.7000,133.8833,20.68,33,0,3.60,AU,1712225333
60,60,general conesa,-40.1000,-64.4333,11.93,49,0,7.06,AR,1712225333
67,67,dubrovnik,42.6481,18.0922,17.80,36,0,4.12,HR,1712225334
82,82,marsa alam,25.0757,34.8918,28.25,49,0,4.80,EG,1712225335
100,100,grants,35.1473,-107.8514,-0.87,59,0,2.06,US,1712225336
152,152,san patricio,28.0170,-97.5169,17.91,41,0,3.85,US,1712225342
167,167,fushe-kruje,41.4783,19.7178,21.08,46,0,2.06,AL,1712225133
280,280,chandler,33.3062,-111.8412,20.25,51,0,3.60,US,1712225175
330,330,el maamoura,36.4650,10.8061,19.63,45,0,1.45,TN,1712225357


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
s = ["City","Country", "Lat", "Lng", "Humidity"]
hotel_df = vacation_data_df[s].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,port elizabeth,ZA,-33.9180,25.5701,40,
50,alice springs,AU,-23.7000,133.8833,33,
60,general conesa,AR,-40.1000,-64.4333,49,
67,dubrovnik,HR,42.6481,18.0922,36,
82,marsa alam,EG,25.0757,34.8918,49,
100,grants,US,35.1473,-107.8514,59,
152,san patricio,US,28.0170,-97.5169,41,
167,fushe-kruje,AL,41.4783,19.7178,46,
280,chandler,US,33.3062,-111.8412,51,
330,el maamoura,TN,36.4650,10.8061,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key, 
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
alice springs - nearest hotel: Aurora Alice Springs
general conesa - nearest hotel: Gran Hotel Conesa
dubrovnik - nearest hotel: Bellevue
marsa alam - nearest hotel: Blue House Hotel
grants - nearest hotel: Southwest Motel
san patricio - nearest hotel: No hotel found
fushe-kruje - nearest hotel: No hotel found
chandler - nearest hotel: Crowne Plaza - San Marcos Golf Resort
el maamoura - nearest hotel: Ben's House
toliara - nearest hotel: Ambary
kristiansund - nearest hotel: Comfort Hotel Fosna
duba - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
48,port elizabeth,ZA,-33.9180,25.5701,40,Waterford Hotel
50,alice springs,AU,-23.7000,133.8833,33,Aurora Alice Springs
60,general conesa,AR,-40.1000,-64.4333,49,Gran Hotel Conesa
67,dubrovnik,HR,42.6481,18.0922,36,Bellevue
82,marsa alam,EG,25.0757,34.8918,49,Blue House Hotel
100,grants,US,35.1473,-107.8514,59,Southwest Motel
152,san patricio,US,28.0170,-97.5169,41,No hotel found
167,fushe-kruje,AL,41.4783,19.7178,46,No hotel found
280,chandler,US,33.3062,-111.8412,51,Crowne Plaza - San Marcos Golf Resort
330,el maamoura,TN,36.4650,10.8061,45,Ben's House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = 2,
    scale = 10,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)